In [1]:
import pdfplumber
from glob import glob
from tqdm import tqdm 
import copy 
import os
import docx
import json
import olefile
import zlib
import struct


import re


## extract text line

In [16]:
pdf = pdfplumber.open('../1.pdf')
pages = pdf.pages
for page in pages:
    text = page.extract_text()
    print(text)

RAG기반 서비스 프레임워크 구축 주간보고
RAG 기반 서비스
프레임워크
구축 주간 보고
마인즈앤컴퍼니
2024.04.16
1
RAG기반 서비스 프레임워크 구축 주간보고
INDEX
I. 마일스톤
II. 프로젝트 진척 현황
III. 인력 투입 현황
IV. 주간 업무 진행 현황
V. 이슈 및 리스크 현황
VI. Action Item 진행 사항
C
ONTENTS
2
RAG기반 서비스 프레임워크 구축 주간보고
마일스톤
1주차(04/08 ~ 04/16)
’24.4 ’24.4 ’24.5 ’24.6 ’24.7
분석/설계 구현(개발) 테스트 이행/안정화
프로젝트 관리
착수보고 중간보고 서비스 최종보고
오픈
데이터 수집 대상
데이터 수집 대상
데이터 전처리 현황 및 데이터 수집 파이프 라인 개발 데이터 수집
요건 정의
인터페이스 분석
및 벡터 DB
구축
데이터 전처리 문단 분리 개발 및 저장
Vector DB 인덱스 저장 단위 테스트 통합 테스트
방안 설계 정보 추출 개발 및 저장
이행 및 안정화
요구사항 분석 상세 요건 설계 LLMOps 솔루션 커스터마이징 개발
배포 모델 운영 안정화
IT Admin
업무 인수인계/
솔루션 설치 및
Integration 테스트 산출물 정리
시스템 통합
요구사항 분석 상세 요건 설계 서비스 관리 도구 솔루션 커스터마이징 개발
현업 Admin
솔루션 설치 및
Integration 테스트
시스템 통합
3
RAG기반 서비스 프레임워크 구축 주간보고
WBS
1주차(04/08 ~ 04/16) - 프로젝트 시작 시점 변경 및 요구사항 구체화로 WBS 현행화 중 (~04.19)
WBS_V2.0 w1~w2
4
RAG기반 서비스 프레임워크 구축 주간보고
솔루션 배포 일정
1주차(04/08 ~ 04/16) – 초기 버전 반입 절차 확인 (~04/19)
’24.4月 ’24.5月 ’24.6月 ’24.7月 ’24.8月
W1 W2 W3 W4 W1 W2 W3 W4 W5 W1 W2 W3 W4 W1 W2 W3 W4 W5 W1 W2 W3 W4
착수보고 중간보고 솔루션 배포 

In [2]:
## 텍스트 추출 및 라인 분리

pattern1 = r'[가-힣]\.'
pattern2 = r'\d\.|[\u2460-\u2473]'

def extract_lines(type, path):

    ## HWP

    if type == 'hwp':

        f = olefile.OleFileIO(path)
        dirs = f.listdir()

        # HWP 파일 검증
        if ["FileHeader"] not in dirs or \
        ["\x05HwpSummaryInformation"] not in dirs:
            raise Exception("Not Valid HWP.")

        # 문서 포맷 압축 여부 확인
        header = f.openstream("FileHeader")
        header_data = header.read()
        is_compressed = (header_data[36] & 1) == 1

        # Body Sections 불러오기
        nums = []
        for d in dirs:
            if d[0] == "BodyText":
                nums.append(int(d[1][len("Section"):]))
        sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

        # 전체 text 추출
        hwp_text = ""
        for section in sections:
            bodytext = f.openstream(section)
            data = bodytext.read()
            if is_compressed:
                unpacked_data = zlib.decompress(data, -15)
            else:
                unpacked_data = data
        
            # 각 Section 내 text 추출    
            section_text = ""
            i = 0
            size = len(unpacked_data)
            while i < size:
                header = struct.unpack_from("<I", unpacked_data, i)[0]
                rec_type = header & 0x3ff
                rec_len = (header >> 20) & 0xfff

                if rec_type in [67]:
                    rec_data = unpacked_data[i+4:i+4+rec_len]
                    section_text += rec_data.decode('utf-16')
                    section_text += "\n"

                i += 4 + rec_len

            hwp_text += section_text
            hwp_text += "\n"

        original_list = hwp_text.split('\n')

        cleaned_list = [text for text in original_list if '\x00' not in text]
        cleaned_list = [text.lstrip().replace('\r','') for text in cleaned_list]

        results2 = []
        for line in cleaned_list:
            result_dict = {'text':''}
            result_dict['text'] += line
            results2.append(result_dict)

        
    
    
    ## DOCX
    elif type == 'docx':
        file_path = path
        try:
            doc = docx.Document(file_path)

            results2 = []
            for para in doc.paragraphs:
                result_dict = {'text':''}
                if para.text != '':
                    result_dict['text'] += para.text
                    results2.append(result_dict)
        except:
            import docx2txt
            doc = docx2txt.process(file_path)
            doc = [line for line in doc.split('\n') if line != '']
            results2 = []
            for line in doc:
                result_dict = {'text':''}
                result_dict['text'] += line
                results2.append(result_dict)

    
    
    
    ## PDF
    elif type == 'pdf':
        pdf = pdfplumber.open(path)
        pages = pdf.pages

        word_list_pages = []
        
        for i in tqdm(range(len(pages))):
            # 이걸 하면 page 에 있는 word 들이 하나하나 나오게 됩니다 dict 형태
            word_list_page = pages[i].extract_words(keep_blank_chars=False, use_text_flow=True, split_at_punctuation=True, expand_ligatures=True, extra_attrs=['fontname','size'])
            
            for j in word_list_page:
            
                j['page'] = i+1
                j['x0'] = int(j['x0'])
                j['x1'] = int(j['x1'])
                j['doctop'] = int(j['doctop'])
                
                #del j['x1']
                del j['upright']
                del j['direction']
                del j['top']
                del j['bottom']
                del j['fontname']
                
            word_list_page_sorted = sorted(word_list_page, key=lambda x : (x['doctop'], x['x0'])) 
            word_list_pages += word_list_page_sorted
            
        results = []
        doctop_v = 0
        result = {}
        high_text, low_text = [],[]
        prev = word_list_pages[0]
        for i in range(len(word_list_pages)):
            tmp = copy.deepcopy(word_list_pages[i])

            text = tmp['text']
            
            if i == 0:
                doctop_v = tmp['doctop']
                result = tmp
                continue
            
            if (tmp['doctop'] != doctop_v):
                # test
                if ((542.0 <= prev['x1'] <= 566.0) and ((tmp['doctop'] - prev['doctop']) <= 20.0)) :
                    if prev['text'][-1] == '.' or prev['text'][-1].isdigit() :
                        result['line_finish'] = 1
                    else:
                        result['line_finish'] = 0

                elif tmp['text']=='제1장':
                    result['line_finish'] = 1
                        
                else:
                    result['line_finish'] = 1
            
                #

                doctop_v = tmp['doctop']
                result['text'] = result['text'].replace(' . .','..').replace('..','')
                results.append(result)
                result = {}
                
            if len(result) == 0 :
                result = tmp
            else:
                result['text'] = result['text'] + ' ' + text

            prev = word_list_pages[i]
            result['x1'] = tmp['x1']

        prev_dict = results[0]
        high_text.append(prev_dict)
        for i in range(len(results)):
            
            tmp = copy.deepcopy(results[i])
            if tmp['page'] > prev_dict['page']:
                low_text.append(prev_dict)
                high_text.append(tmp)
            
            prev_dict = results[i]
        
        for i in range(len(low_text)):
            results.remove(low_text[i])
        for i in range(len(high_text)):
            results.remove(high_text[i])


        ## 끝나지 않은 line을 한 문장으로 잇기
        results2=[]
        tmp_dict={'text':''}
        for i in range(len(results)):
            tmp_line = results[i]
            if tmp_line['line_finish'] == 0:
                if tmp_dict['text'] == '':
                    tmp_dict = tmp_line
                    tmp_dict['start_doctop'] = tmp_dict['doctop']
                    del tmp_dict['doctop']
                else:
                    tmp_dict['text'] = tmp_dict['text'] + ' ' + tmp_line['text']
                    tmp_dict['x1'] = tmp_line['x1']
            else:
                if tmp_dict['text'] == '':
                    tmp_line['start_doctop'] = tmp_line['doctop']
                    tmp_line['end_doctop'] = tmp_line['doctop']
                    del tmp_line['doctop']
                    results2.append(tmp_line)
                else:
                    tmp_dict['text'] = tmp_dict['text'] + ' ' + tmp_line['text']
                    tmp_dict['end_doctop'] = tmp_line['doctop']
                    tmp_dict['x1'] = tmp_line['x1']
                    results2.append(tmp_dict)
                    tmp_dict={'text':''}

    results3=[]

    for i in range(len(results2)):
        tmp = results2[i]
        if tmp['text'].split(' ')[0].startswith('제'):
            if tmp['text'].split(' ')[0].endswith('편'):
                tmp['title'] = '제n편'
            
            elif tmp['text'].split(' ')[0].endswith('장'):
                tmp['title'] = '제n장'

            elif tmp['text'].split(' ')[0].endswith('절'):
                tmp['title'] = '제n절'

            else: 
                tmp['title'] = '제n조'

        else:
            if i == 0:
                tmp['title'] = '대제목'

            else:
                string = tmp['text'].split(' ')[0]
                if re.match(pattern1, string):
                    tmp['title'] = '본문_가나다'

                elif re.match(pattern2, string):
                    tmp['title'] = '본문_숫자'

                else:
                    tmp['title'] = '본문_기타'

        results3.append(tmp)

    return results3
        




## title 지정

In [3]:
# 라인 별 제목 분류

def get_title(input_list):
    #pattern = r'(제\d+조 \( .+? \))|(제\d+조의\d+ \( .+? \))|(제\d +- \d+조 \( .+? \))|(<[^>]+>)|(제\d+조\(.+?\))'
    pattern = r'(제\d+조 \(.*?\))|(제\d+조의\d+ \(.*?\))|(제\d+-\d+조 \(.*?\))|(<[^>]+>)|(제\d+조\(.*?\))'
    #제1조 (뭐시기) | 제1조의2 (뭐시기) | 제1-1조 (뭐시기) | 
    
    results = []
    temp_title={'title_0':'','title_1':'','title_2':'','title_3':'','title_4':''} # 대제목, 제n편, 제n장, 제n절, 제n조,
    for i in range(len(input_list)):
        temp_dict = input_list[i]
        
        
        if temp_dict['title'] == '대제목' :
            if temp_title['title_0'] != temp_dict['text']:
                temp_title['title_0'] = temp_dict['text']
                temp_title['title_1'] = ''
                temp_title['title_2'] = ''
                temp_title['title_3'] = ''
                temp_title['title_4'] = ''
                
            temp_dict['title_detail'] = temp_title['title_0']
        
        if temp_dict['title'] == '제n편':
            if temp_title['title_1'] != temp_dict['text']:
                temp_title['title_1'] = temp_dict['text']
                temp_title['title_2'] = ''
                temp_title['title_3'] = ''
                temp_title['title_4'] = ''
                
            temp_dict['title_detail'] = temp_title['title_0'] + ' ' + temp_title['title_1']
            
        
        if temp_dict['title'] == '제n장':
            if temp_title['title_2'] != temp_dict['text']:
                temp_title['title_2'] = temp_dict['text']
                temp_title['title_3'] = ''
                temp_title['title_4'] = ''
                
         
            temp_dict['title_detail'] = temp_title['title_0'] + ' ' +temp_title['title_1'] + ' ' + temp_title['title_2']

        if temp_dict['title'] == '제n절':
            if temp_title['title_3'] != temp_dict['text']:
                temp_title['title_3'] = temp_dict['text']
                temp_title['title_4'] = ''
                
         
            temp_dict['title_detail'] = temp_title['title_0'] + ' ' + temp_title['title_1'] + ' ' + temp_title['title_2'] + ' ' + temp_title['title_3']

        if temp_dict['title'] == '제n조':    
            matches= re.findall(pattern, temp_dict['text'])
            output = ''
            for match in matches:
                output += [x for x in match if x != ''][0] + ' '
            
            if temp_title['title_4'] !=  output:
                temp_title['title_4'] = output
            
            temp_dict['title_detail'] = temp_title['title_0'] + ' ' + temp_title['title_1'] + ' ' + temp_title['title_2'] + ' ' + temp_title['title_3'] + ' ' + temp_title['title_4']


        if temp_dict['title'] == '본문_기타' or '본문_숫자'  :
            temp_dict['title_detail'] = temp_title['title_0'] + ' ' + temp_title['title_1'] + ' ' + temp_title['title_2'] + ' ' + temp_title['title_3'] + ' ' + temp_title['title_4']
                
            
                
        temp_dict['line_length'] = len(temp_dict['text'])


        results.append(temp_dict)

    return results


## 청크 생성

In [4]:
# 청크 생성

def get_paragraph(input_list):
    results = []
    prev_dict = input_list[0]
    result = {'par_text':'','title_detail':prev_dict['title_detail'],'is_split' : 0}
    
    for i in range(1,len(input_list)):
        temp_dict = input_list[i]
        result['par_text'] = result['par_text'] + ' ' + prev_dict['text']

        if (temp_dict['title_detail']!= prev_dict['title_detail']): 
            result['length'] = len(result['par_text'])
            
            if results != [] and (results[-1]['title_detail'] == prev_dict['title_detail']):
                result['is_split'] = 1
            else:
                result['is_split'] = 0
            results.append(result)
            result = {'par_text':'','title_detail':temp_dict['title_detail']}

        if len(result['par_text']) + temp_dict['line_length'] > 600:
            result['length'] = len(result['par_text'])
            result['is_split'] = 1
            
            results.append(result)
            result = {'par_text':'','title_detail':temp_dict['title_detail'],'is_split':1}

        prev_dict = input_list[i]
        if result['title_detail'] == input_list[-1]['title_detail'] and result['par_text'] == '':
            result['par_text'] = result['par_text'] + ' ' + prev_dict['text']
        
        
    result['length'] = len(result['par_text'])
    result['is_split'] = 0
    results.append(result)

    return results
        

## json으로 저장

In [12]:
def test(type,file_path):
    lines_list = extract_lines(type, file_path)
        
    title_list = get_title(lines_list)
    paragraph_list = get_paragraph(title_list)
    

    
    
    return paragraph_list

In [7]:
def file2json(type ,path):
    for file_path in path:


        # 저장 경로 설정
        # ./data/out_law_docx/클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률 시행령(대통령령)(제33220호)(20230112).docx 
        # ->>  ./data/chunk/docx/클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률 시행령(대통령령)(제33220호)(20230112).json

        json_file_path = file_path.replace(f'out_law_{type}',f'chunk/{type}').replace(f'.{type}','.json')

        
           
        lines_list = extract_lines(type, file_path)
        
        title_list = get_title(lines_list)
        paragraph_list = get_paragraph(title_list)
        

        
        # with open(json_file_path, 'w', encoding='utf-8') as json_output:
        #     json.dump({'content': paragraph_list}, json_output, ensure_ascii=False, indent=4)
        return paragraph_list
        print(f'{file_path} 파일을 {json_file_path}로 변환 및 저장했습니다.')

In [14]:
test('pdf', '../1.pdf')

100%|██████████| 17/17 [00:00<00:00, 33.57it/s]


[{'par_text': ' RAG 기반 서비스 프레임워크 구축 주간 보고 마인즈앤컴퍼니 2024 . 04 . 16 INDEX I . 마일스톤 II . 프로젝트 진척 현황 III . 인력 투입 현황 IV . 주간 업무 진행 현황 V . 이슈 및 리스크 현황 VI . Action Item 진행 사항 C ONTENTS 마일스톤 1 주차 ( 04 / 08 ~ 04 / 16 ) ’24 . 4 ’24 . 4 ’24 . 5 ’24 . 6 ’24 . 7 테스트 분석 / 설계 구현 ( 개발 ) 이행 / 안정화 프로젝트 관리 착수보고 중간보고 서비스 최종보고 오픈 데이터 수집 대상 데이터 수집 대상 데이터 수집 현황 및 데이터 수집 파이프 라인 개발 데이터 전처리 요건 정의 인터페이스 분석 및 벡터 DB 구축 데이터 전처리 문단 분리 개발 및 저장 Vector DB 인덱스 저장 단위 테스트 통합 테스트 방안 설계 정보 추출 개발 및 저장 이행 및 안정화 요구사항 분석 상세 요건 설계 LLMOps 솔루션 커스터마이징 개발 배포 모델 운영 안정화 IT Admin 업무 인수인계 / 솔루션 설치 및 산출물 정리 Integration 테스트 시스템 통합',
  'title_detail': 'RAG 기반 서비스    ',
  'is_split': 1,
  'length': 568},
 {'par_text': ' 요구사항 분석 상세 요건 설계 서비스 관리 도구 솔루션 커스터마이징 개발 요구사항 분석 상세 요건 설계 서비스 관리 도구 솔루션 커스터마이징 개발 현업 Admin 솔루션 설치 및 Integration 테스트 시스템 통합 WBS 1 주차 ( 04 / 08 ~ 04 / 16 ) - 프로젝트 시작 시점 변경 및 요구사항 구체화로 WBS 현행화 중 ( ~ 04 . 19 ) WBS _ V2 . 0 w1 ~ w2 솔루션 배포 일정 1 주차 ( 04 / 08 ~ 04 / 16 ) – 초기 버전 반입 절차 확인 ( ~ 04 / 19 ) ’24 . 4 月 ’24 . 5 月 ’24 . 6 月 ’2

In [6]:
#pdf_path = glob('./data/out_law_pdf/*.pdf')
docx_path = glob('./data/out_law_docx/*.docx')
#hwp_path = ['./data/금융회사 지배구조 감독규정(금융위원회고시)(제2018-4호)(20180401).hwp']#/Users/imhyeongbin/Desktop/미래에셋/data/금융회사 지배구조 감독규정(금융위원회고시)(제2018-4호)(20180401).hwp

#file2json('pdf',pdf_path)
file2json('docx',docx_path)
#file2json('hwp',hwp_path)

./data/out_law_docx/클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률 시행령(대통령령)(제33220호)(20230112).docx 파일을 ./data/chunk/docx/클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률 시행령(대통령령)(제33220호)(20230112).json로 변환 및 저장했습니다.
./data/out_law_docx/안녕하세요1.docx 파일을 ./data/chunk/docx/안녕하세요1.json로 변환 및 저장했습니다.
./data/out_law_docx/전자서명법 시행령(대통령령)(제33723호)(20230915).docx 파일을 ./data/chunk/docx/전자서명법 시행령(대통령령)(제33723호)(20230915).json로 변환 및 저장했습니다.
./data/out_law_docx/전자금융거래법 시행령(대통령령)(제33913호)(20231212).docx 파일을 ./data/chunk/docx/전자금융거래법 시행령(대통령령)(제33913호)(20231212).json로 변환 및 저장했습니다.
./data/out_law_docx/전자서명법(법률)(제18479호)(20221020).docx 파일을 ./data/chunk/docx/전자서명법(법률)(제18479호)(20221020).json로 변환 및 저장했습니다.
./data/out_law_docx/증권의 발행 및 공시 등에 관한 규정 시행세칙(금융감독원세칙)(20160630).docx 파일을

In [32]:
import re
pattern = r'(제\d+조 \(.*?\))|(제\d+조의\d+ \(.*?\))|(제\d+-\d+조 \(.*?\))|(<[^>]+>)|(제\d+조\(.*?\))'

x = '제1조 (목적) ㄴㅇㄹ'

matches = re.findall(pattern, x)

# 추출된 매칭 부분 중 비어 있지 않은 항목만 필터링
extracted = [match for match in matches if match]

print(extracted)

['제1조 (목적)']


In [55]:
## 텍스트 추출 및 라인 분리

pattern1 = r'[가-힣]\.'
pattern2 = r'\d\.|[\u2460-\u2473]'

def extract_lines(type, path):

    ## HWP

    if type == 'hwp':

        f = olefile.OleFileIO(path)
        dirs = f.listdir()

        # HWP 파일 검증
        if ["FileHeader"] not in dirs or \
        ["\x05HwpSummaryInformation"] not in dirs:
            raise Exception("Not Valid HWP.")

        # 문서 포맷 압축 여부 확인
        header = f.openstream("FileHeader")
        header_data = header.read()
        is_compressed = (header_data[36] & 1) == 1

        # Body Sections 불러오기
        nums = []
        for d in dirs:
            if d[0] == "BodyText":
                nums.append(int(d[1][len("Section"):]))
        sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

        # 전체 text 추출
        hwp_text = ""
        for section in sections:
            bodytext = f.openstream(section)
            data = bodytext.read()
            if is_compressed:
                unpacked_data = zlib.decompress(data, -15)
            else:
                unpacked_data = data
        
            # 각 Section 내 text 추출    
            section_text = ""
            i = 0
            size = len(unpacked_data)
            while i < size:
                header = struct.unpack_from("<I", unpacked_data, i)[0]
                rec_type = header & 0x3ff
                rec_len = (header >> 20) & 0xfff

                if rec_type in [67]:
                    rec_data = unpacked_data[i+4:i+4+rec_len]
                    section_text += rec_data.decode('utf-16')
                    section_text += "\n"

                i += 4 + rec_len

            hwp_text += section_text
            hwp_text += "\n"

        original_list = hwp_text.split('\n')

        cleaned_list = [text for text in original_list if '\x00' not in text]
        cleaned_list = [text.lstrip().replace('\r','') for text in cleaned_list]

        results2 = []
        for line in cleaned_list:
            result_dict = {'text':''}
            result_dict['text'] += line
            results2.append(result_dict)

        
    
    
    ## DOCX
    elif type == 'docx':
        file_path = path
        try:
            doc = docx.Document(file_path)

            results2 = []
            for para in doc.paragraphs:
                result_dict = {'text':''}
                if para.text != '':
                    result_dict['text'] += para.text
                    results2.append(result_dict)
        except:
            import docx2txt
            doc = docx2txt.process(file_path)
            doc = [line for line in doc.split('\n') if line != '']
            results2 = []
            for line in doc:
                result_dict = {'text':''}
                result_dict['text'] += line
                results2.append(result_dict)

    
    
    
    ## PDF
    elif type == 'pdf':
        pdf = pdfplumber.open(path)
        pages = pdf.pages

        word_list_pages = []
        
        for i in tqdm(range(len(pages))):
            # 이걸 하면 page 에 있는 word 들이 하나하나 나오게 됩니다 dict 형태
            word_list_page = pages[i].extract_words(keep_blank_chars=False, use_text_flow=True, split_at_punctuation=True, expand_ligatures=True, extra_attrs=['fontname','size'])
            
            for j in word_list_page:
            
                j['page'] = i+1
                j['x0'] = int(j['x0'])
                j['x1'] = int(j['x1'])
                j['doctop'] = int(j['doctop'])
                
                #del j['x1']
                del j['upright']
                del j['direction']
                del j['top']
                del j['bottom']
                del j['fontname']
                
            word_list_page_sorted = sorted(word_list_page, key=lambda x : (x['doctop'], x['x0'])) 
            word_list_pages += word_list_page_sorted
            
        results = []
        doctop_v = 0
        result = {}
        high_text, low_text = [],[]
        prev = word_list_pages[0]
        for i in range(len(word_list_pages)):
            tmp = copy.deepcopy(word_list_pages[i])

            text = tmp['text']
            
            if i == 0:
                doctop_v = tmp['doctop']
                result = tmp
                continue
            
            if (tmp['doctop'] != doctop_v):
                # test
                if ((542.0 <= prev['x1'] <= 566.0) and ((tmp['doctop'] - prev['doctop']) <= 20.0)) :
                    if prev['text'][-1] == '.' or prev['text'][-1].isdigit() :
                        result['line_finish'] = 1
                    else:
                        result['line_finish'] = 0

                elif tmp['text']=='제1장':
                    result['line_finish'] = 1
                        
                else:
                    result['line_finish'] = 1
            
                #

                doctop_v = tmp['doctop']
                result['text'] = result['text'].replace(' . .','..').replace('..','')
                results.append(result)
                result = {}
                
            if len(result) == 0 :
                result = tmp
            else:
                result['text'] = result['text'] + ' ' + text

            prev = word_list_pages[i]
            result['x1'] = tmp['x1']

        prev_dict = results[0]
        high_text.append(prev_dict)
        for i in range(len(results)):
            
            tmp = copy.deepcopy(results[i])
            if tmp['page'] > prev_dict['page']:
                low_text.append(prev_dict)
                high_text.append(tmp)
            
            prev_dict = results[i]
        
        for i in range(len(low_text)):
            results.remove(low_text[i])
        for i in range(len(high_text)):
            results.remove(high_text[i])


        ## 끝나지 않은 line을 한 문장으로 잇기
        results2=[]
        tmp_dict={'text':''}
        for i in range(len(results)):
            tmp_line = results[i]
            if tmp_line['line_finish'] == 0:
                if tmp_dict['text'] == '':
                    tmp_dict = tmp_line
                    tmp_dict['start_doctop'] = tmp_dict['doctop']
                    del tmp_dict['doctop']
                else:
                    tmp_dict['text'] = tmp_dict['text'] + ' ' + tmp_line['text']
                    tmp_dict['x1'] = tmp_line['x1']
            else:
                if tmp_dict['text'] == '':
                    tmp_line['start_doctop'] = tmp_line['doctop']
                    tmp_line['end_doctop'] = tmp_line['doctop']
                    del tmp_line['doctop']
                    results2.append(tmp_line)
                else:
                    tmp_dict['text'] = tmp_dict['text'] + ' ' + tmp_line['text']
                    tmp_dict['end_doctop'] = tmp_line['doctop']
                    tmp_dict['x1'] = tmp_line['x1']
                    results2.append(tmp_dict)
                    tmp_dict={'text':''}

    results3=[]

    for i in range(len(results2)):
        tmp = results2[i]
        if tmp['text'].split(' ')[0].startswith('제'):
            if tmp['text'].split(' ')[0].endswith('편'):
                tmp['title'] = '제n편'
            
            elif tmp['text'].split(' ')[0].endswith('장'):
                tmp['title'] = '제n장'

            elif tmp['text'].split(' ')[0].endswith('절'):
                tmp['title'] = '제n절'

            else: 
                tmp['title'] = '제n조'

        else:
            if i == 0:
                tmp['title'] = '대제목'

            else:
                string = tmp['text'].split(' ')[0]
                if re.match(pattern1, string):
                    tmp['title'] = '본문_가나다'

                elif re.match(pattern2, string):
                    tmp['title'] = '본문_숫자'

                else:
                    tmp['title'] = '본문_기타'

        results3.append(tmp)

    return results3
        

def get_title(input_list):
    #pattern = r'(제\d+조 \( .+? \))|(제\d+조의\d+ \( .+? \))|(제\d +- \d+조 \( .+? \))|(<[^>]+>)|(제\d+조\(.+?\))'
    pattern = r'(제\d+조 \(.*?\))|(제\d+조의\d+ \(.*?\))|(제\d+-\d+조 \(.*?\))|(<[^>]+>)|(제\d+조\(.*?\))'
    #제1조 (뭐시기) | 제1조의2 (뭐시기) | 제1-1조 (뭐시기) | 
    
    results = []
    
    for i in range(len(input_list)):
        md_dict={'role':'','contents':''} 
        temp_dict = input_list[i]
        
        
        if temp_dict['title'] in ['대제목','제n편','제n장','제n절'] :
            md_dict['role'] = 'title'
            md_dict['contents'] = temp_dict['text']
            

                
            
        elif temp_dict['title'] == '제n조':    
            matches= re.findall(pattern, temp_dict['text'])
            output = ''
            for match in matches:
                output += [x for x in match if x != ''][0]
            
            md_dict_1 = md_dict.copy()
            md_dict_2 = md_dict.copy()
            md_dict_1['role'] = 'sectionHeading'
            md_dict_1['contents'] = output
            results.append(md_dict_1)

            md_dict_2['role'] = 'context'
            md_dict_2['contents'] = temp_dict['text'].split(output)[1]
            results.append(md_dict_2)
            

        else:
            md_dict['role'] = 'context'
            md_dict['contents'] = temp_dict['text']
        
        if md_dict['contents'] != '':
            results.append(md_dict)
        

    return results


        




In [56]:
pattern = r'(제\d+조 \(.*?\))|(제\d+조의\d+ \(.*?\))|(제\d+-\d+조 \(.*?\))|(<[^>]+>)|(제\d+조\(.*?\))'
text = '제1조(목적) 이 규정은 「금융소비자 보호에 관한 법률」 및 같은 법 <별표 1> 시행령에서 위임하는 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.'
matches= re.findall(pattern, text)
output = ''
for match in matches:
    output += [x for x in match if x != ''][0]
print(output)

제1조(목적)<별표 1>


In [57]:
lines_list = extract_lines('pdf','./data/out_law_pdf/금융소비자 보호에 관한 감독규정(금융위원회고시)(제2023-38호)(20230705).pdf')
titles_list = get_title(lines_list)
titles_list

100%|██████████| 26/26 [00:00<00:00, 31.39it/s]


[{'role': 'title', 'contents': '금융소비자 보호에 관한 감독규정'},
 {'role': 'context',
  'contents': '[ 시행 2023 . 7 . 5 . ] [ 금융위원회고시 제2023 - 38호 , 2023 . 7 . 5 . , 일부개정 ]'},
 {'role': 'context', 'contents': '금융위원회 ( 금융소비자정책과 ) , 02 - 2100 - 2633'},
 {'role': 'sectionHeading', 'contents': '제1조 ( 목적 )'},
 {'role': 'context',
  'contents': ' 이 규정은 「금융소비자 보호에 관한 법률」 및 같은 법 시행령에서 위임하는 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다 .'},
 {'role': 'sectionHeading', 'contents': '제2조 ( 정의 )'},
 {'role': 'context',
  'contents': ' ① 「금융소비자 보호에 관한 법률 시행령」 ( 이하 " 영 " 이라 한다 ) 제2조제1항제7호에서 " 금융위원회가 정하여 고시하는 것 " 이란 다음 각 호의 어느 하나에 해당하는 것을 말한다 .'},
 {'role': 'context',
  'contents': '1 . 다음 각 목의 자가 계약에 따라 금융소비자로부터 금전을 받고 장래에 그 금전과 그에 따른 이자 등의 대가를 지급하기로 하는 계약 . 다만 , 「주택법」에 따른 입주자저축은 제외한다 .'},
 {'role': 'context',
  'contents': '가 . 「금융산업의 구조개선에 관한 법률」에 따라 「자본시장과 금융투자업에 관한 법률」에 따른 종합금융회사 와 합병한 기관 ( 「예금자보호법」 제2조제1호가목부터 사목까지의 부보금융회사를 말한다 )'},
 {'role': 'context', 'contents': '나 . 「농업협동조합법」에 따른 농협은행'},
 {'role': 'context', 'contents': 

In [25]:
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'

doc = docx.Document(file_path)


In [26]:


def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of either Table or Paragraph.
    """
    print(type(parent))
    if isinstance(parent, docx.document.Document):
        parent_elm = doc.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iter():
        if isinstance(child, docx.oxml.text.paragraph.CT_P):
            if child.text != '':
                yield ("(paragraph)", child,child.text)

        elif isinstance(child, docx.oxml.table.CT_Tc):
            yield ("(table)", child,child.text)

for i in iter_block_items(doc): 
    print(i)

<class 'docx.document.Document'>
('(paragraph)', <CT_P '<w:p>' at 0x10d13a360>, '안녕하세요1')
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '안녕하세요2')
('(paragraph)', <CT_P '<w:p>' at 0x10d12fea0>, '안녕하세요3ㄴ')
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '안녕하세요4')
('(paragraph)', <CT_P '<w:p>' at 0x10d139f90>, '안녕하세요5')
('(table)', <CT_Tc '<w:tc>' at 0x10d040900>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fea0>, '안녕하세요6')
('(table)', <CT_Tc '<w:tc>' at 0x10d13a360>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '안녕하세요7')
('(table)', <CT_Tc '<w:tc>' at 0x10d128e50>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d040900>, '안녕하세요8')
('(table)', <CT_Tc '<w:tc>' at 0x10d139f90>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fea0>, '안녕하세요9')
('(table)', <CT_Tc '<w:tc>' at 0x10d040900>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d128e50>, '안녕하세요10')
('(table)', <CT_Tc '<w:tc>' at 0x10d040900>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fea0>, '안녕하세요11')
('(table)', <CT_Tc '<w:tc>' at 

In [18]:
import json
import docx
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'

doc = docx.Document(file_path)


dict_list = []


    



def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of either Table or Paragraph.
    """
    print(type(parent))
    if isinstance(parent, docx.document.Document):
        parent_elm = doc.element.body
       
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
        #parent_elm = parent.table
        
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iter():
        if isinstance(child, docx.oxml.text.paragraph.CT_P):
            yield ("(paragraph)", child,child.text)
        elif isinstance(child, docx.oxml.table.CT_Tc):
            yield ("(table)", child,child.text)
        
    

table_list = []
paragraph_list = []


for i in iter_block_items(doc): 
    print(i)
    



<class 'docx.document.Document'>
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '안녕하세요1')
('(paragraph)', <CT_P '<w:p>' at 0x10d13c9a0>, '안녕하세요2')
('(paragraph)', <CT_P '<w:p>' at 0x10d13a360>, '안녕하세요3ㄴ')
('(paragraph)', <CT_P '<w:p>' at 0x10d12fef0>, '안녕하세요4')
('(paragraph)', <CT_P '<w:p>' at 0x10d13c9a0>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13a360>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13c9a0>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13a360>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13aea0>, '')
('(paragraph)', <CT_P '<w:p>' at 0x10d13c9a0>, '안녕하세요5')
('(paragraph)', <CT_P '<w:p>' at 0x10d12fc20>, '')
('(table)', <CT_Tc '<w:tc>' at 0x10d13a360>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fc20>, '안녕하세요6')
('(table)', <CT_Tc '<w:tc>' at 0x10d13c9a0>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fc20>, '안녕하세요7')
('(table)', <CT_Tc '<w:tc>' at 0x10d13aea0>, None)
('(paragraph)', <CT_P '<w:p>' at 0x10d12fc20>, '안녕하세요8')


In [52]:
for i, e in enumerate(doc.element.iter()):
    print(i,e,type(e))
    print(e.tag)

0 <CT_Document '<w:document>' at 0x10d13c6d0> <class 'docx.oxml.document.CT_Document'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}document
1 <CT_Body '<w:body>' at 0x10d127220> <class 'docx.oxml.document.CT_Body'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}body
2 <CT_P '<w:p>' at 0x10814c360> <class 'docx.oxml.text.paragraph.CT_P'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p
3 <CT_R '<w:r>' at 0x10d1246d0> <class 'docx.oxml.text.run.CT_R'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}r
4 <CT_RPr '<w:rPr>' at 0x10d134ea0> <class 'docx.oxml.text.font.CT_RPr'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rPr
5 <CT_Fonts '<w:rFonts>' at 0x10d134f90> <class 'docx.oxml.text.font.CT_Fonts'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rFonts
6 안녕하세요1 <class 'docx.oxml.text.run.CT_Text'>
{http://schemas.openxmlformats.org/wordprocessingml/2006/main}t
7 <CT_P '<w:p>' at 0x10d1280e0> <cla

In [1]:
import docx
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'

doc = docx.Document(file_path)
table_text_list=[]
para_text_list=[]
block_list = []
table_dict={'table_idx':0,'table_text':''}

for e in doc.element.iter():
    block_list.append(e)

prev_block= block_list[0]
isCell = 0
for i in range(1, len(block_list)):
    temp_block = block_list[i]
    
    if type(prev_block) == docx.oxml.text.run.CT_Text:
        

        if type(temp_block) == docx.oxml.table.CT_Tc: # table
            table_dict['table_text'] = prev_block.text
            table_text_list.append(table_dict)
            isCell = 1

        else:
            para_text_list.append(prev_block.text) # paragraph
            if isCell == 1:
                isCell=0
                table_dict['table_idx'] += 1

        print(prev_block.text,isCell)
            
    prev_block = temp_block

print('-------')
print(para_text_list)
print('-----')
print(table_text_list)



    

'''
<CT_P '<w:p>' at 0x139383950>
7 <CT_R '<w:r>' at 0x139383a40>
8 <CT_RPr '<w:rPr>' at 0x139383c70>
9 <CT_Fonts '<w:rFonts>' at 0x1393831d0>
10 안녕하세요
11 <CT_R '<w:r>' at 0x1393832c0>
12 <CT_RPr '<w:rPr>' at 0x139383c20>
13 <CT_Fonts '<w:rFonts>' at 0x139390770>
14 2'''

안녕하세요1 0
안녕하세요2 0
안녕하세요3 0
ㄴ 0
안녕하세요4 0
안녕하세요5 0
안녕하세요6 1
안녕하세요7 1
안녕하세요8 1
안녕하세요9 1
안녕하세요11 1
안녕하세요11 1
안녕하세요11 1
안녕하세요11 1
안녕하세요12 1
안녕하세요12 1
안녕하세요12 1
안녕하세요12 1
표 0
안녕하세요 0
888 0
안녕하세요11 1
안녕하세요8 1
안녕하세요9 1
안녕하세요10 0
안녕하세요11 1
안녕하세요11 1
안녕하세요11 0
안녕하세요12 1
안녕하세요12 1
안녕하세요12 1
안녕하세요12 0
안녕하세요 13 0
-------
['안녕하세요1', '안녕하세요2', '안녕하세요3', 'ㄴ', '안녕하세요4', '안녕하세요5', '표', '안녕하세요', '888', '안녕하세요10', '안녕하세요11', '안녕하세요12', '안녕하세요 13']
-----
[{'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'table_idx': 4, 'table_text': '안녕하세요12'}, {'

"\n<CT_P '<w:p>' at 0x139383950>\n7 <CT_R '<w:r>' at 0x139383a40>\n8 <CT_RPr '<w:rPr>' at 0x139383c70>\n9 <CT_Fonts '<w:rFonts>' at 0x1393831d0>\n10 안녕하세요\n11 <CT_R '<w:r>' at 0x1393832c0>\n12 <CT_RPr '<w:rPr>' at 0x139383c20>\n13 <CT_Fonts '<w:rFonts>' at 0x139390770>\n14 2"

In [4]:
'abcdefg'.find('c')

2

In [6]:
'abcdefg'[2:]

'cdefg'

In [45]:
type(list[229])

docx.oxml.text.run.CT_Text

In [38]:
type(list[230])

docx.oxml.table.CT_Tc

In [29]:
list[243]

<CT_Text '<w:t>' at 0x10d1558b0>

In [30]:
list[269]

<CT_Text '<w:t>' at 0x10d12fea0>

In [27]:
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'

doc = docx.Document(file_path)
table_content = []
for table in doc.tables:
    text = '<table>\n'
    for row in table.rows:
        text +='\t<tr>\n'
        for cell in row.cells:
            for para in cell.paragraphs:
                text +='\t\t<td>'
                text += para.text
                text +='</td>\n'
        text +='\t</tr>\n'
    text +='</table>'
    table_content.append(text)
    print(text)

<table>
	<tr>
		<td>안녕하세요6</td>
		<td>안녕하세요7</td>
		<td>안녕하세요8</td>
		<td>안녕하세요9</td>
		<td></td>
	</tr>
	<tr>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
		<td></td>
	</tr>
	<tr>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
		<td></td>
	</tr>
</table>
<table>
	<tr>
		<td>안녕하세요888</td>
		<td>안녕하세요8</td>
		<td>안녕하세요9</td>
		<td>안녕하세요10</td>
	</tr>
	<tr>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
		<td>안녕하세요11</td>
	</tr>
	<tr>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
		<td>안녕하세요12</td>
	</tr>
</table>


In [ ]:
# from docx import Document

# def parse_docx(file_path):
#     document = Document(file_path)
#     elements = []

#     for element in document.element.body:
#         if element.tag.endswith('p'):  # paragraph
#             paragraph = element
#             text = paragraph.text
#             elements.append(('paragraph', text))
#         elif element.tag.endswith('tbl'):  # table
#             table = element
#             table_data = []
#             for row in table.xpath('.//w:tr'):
#                 row_data = []
#                 for cell in row.xpath('.//w:tc'):
#                     cell_text = '   '.join(cell.itertext())
#                     #print("Cell DATA:", cell_text)
#                     row_data.append(cell_text)
#                 table_data.append(row_data)
#             elements.append(('table', table_data))

#     return elementsfile_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'
# parsed_elements = parse_docx(file_path)


# # 파일 경로를 지정해줍니다.

# # 결과를 출력해봅니다.
# for element_type, content in parsed_elements:
#     if element_type == 'paragraph':
#         print("Paragraph:", content)
#     elif element_type == 'table':
#         print("Table:")
#         for row in content:
#             print(row)

In [ ]:
# from docx import Document

# def parse_docx(file_path):
#     document = Document(file_path)
#     elements = []

#     for element in document.element.body:
#         if element.tag.endswith('p'):  # paragraph
#             paragraph = element
#             text = ''.join([node.text for node in paragraph.iter() if node.text])
#             elements.append(('paragraph', text))
#         elif element.tag.endswith('tbl'):  # table
#             table = element
#             table_data = []
#             for row in table.xpath('.//w:tr'):
#                 row_data = []
#                 for cell in row.xpath('.//w:tc'):
#                     cell_texts = [node.text for node in cell.iter() if node.text]
#                     cell_text = ''.join(cell_texts)
#                     row_data.append(cell_text)
#                 table_data.append(row_data)
#             elements.append(('table', table_data))

#     return elements

# # 파일 경로를 지정해줍니다.
# #file_path = 'your_doc_file.docx'
# parsed_elements = parse_docx(file_path)

# # 결과를 출력해봅니다.
# for element_type, content in parsed_elements:
#     if element_type == 'paragraph':
#         print("Paragraph:", content)
#     elif element_type == 'table':
#         print("Table:")
#         for row in content:
#             print(row)

In [8]:
from docx import Document
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'


def parse_docx(file_path):
    document = Document(file_path)
    elements = []

    for element in document.element.body:
        if element.tag.endswith('p'):  # paragraph
            texts = []
            for node in element.iter():
                if node.text and node.text not in texts:
                    texts.append(node.text)
            text = ''.join(texts)
            elements.append(('paragraph', text))
        elif element.tag.endswith('tbl'):  # table
            table_data = []
            for row in element.xpath('.//w:tr'):
                row_data = []
                for cell in row.xpath('.//w:tc'):
                    cell_texts = []
                    for paragraph in cell.xpath('.//w:p'):
                        for node in paragraph.iter():
                            if node.text and (node.text).strip() not in cell_texts:
                                cell_texts.append((node.text).strip())
                    cell_text = ''.join(cell_texts)
                    row_data.append(cell_text)
                table_data.append(row_data)
            elements.append(('table', table_data))

    return elements

# 파일 경로를 지정해줍니다.
#file_path = 'your_doc_file.docx'
parsed_elements = parse_docx(file_path)

# 결과를 출력해봅니다.
for element_type, content in parsed_elements:
    if element_type == 'paragraph':
        print("Paragraph:", content)
    elif element_type == 'table':
        print("Table:")
        for row in content:
            print(row)

Paragraph: 안녕하세요1
Paragraph: 안녕하세요2
Paragraph: 안녕하세요3ㄴ안녕하세요3ㄴ
Paragraph: 안녕하세요4
Paragraph: 
Paragraph: 
Paragraph: 
Paragraph: 
Paragraph: 
Paragraph: 
Paragraph: 안녕하세요5
Paragraph: 
Table:
['안녕하세요6', '안녕하세요7', '안녕하세요8', '안녕하세요9', '']
['안녕하세요11', '안녕하세요11', '안녕하세요11', '안녕하세요11', '']
['안녕하세요12', '안녕하세요12', '안녕하세요12', '안녕하세요12', '']
Paragraph: 
Paragraph: 표
Paragraph: 
Table:
['안녕하세요888안녕하세요888안녕하세요11', '안녕하세요8', '안녕하세요9', '안녕하세요10']
['', '안녕하세요11', '안녕하세요11', '안녕하세요11']
['안녕하세요12', '안녕하세요12', '안녕하세요12', '안녕하세요12']
Paragraph: 
Paragraph: 안녕하세요 13


In [2]:
from docx import Document
file_path = '/Users/imhyeongbin/Desktop/미래에셋/data/out_law_docx/안녕하세요1.docx'


def parse_docx(file_path):
    document = Document(file_path)
    elements = []

    for element in document.element.body:
        if element.tag.endswith('p'):  # paragraph
            texts = []
            for node in element.iter():
                if node.text and node.text not in texts:
                    texts.append(node.text)
            text = ''.join(texts)
            elements.append(('paragraph', text))
        elif element.tag.endswith('tbl'):  # table
            print("sdadsd")
            table = document.tables[document.element.body.index(element)]
            table_data = []
            for row in table.rows:
                row_data = []
                for cell in row.cells:
                    cell_text = cell.text.strip()
                    row_data.append(cell_text)
                table_data.append(row_data)
            elements.append(('table', table_data))
            #         for paragraph in cell.xpath('.//w:p'):
            #             for node in paragraph.iter():
            #                 if node.text and (node.text).strip() not in cell_texts:
            #                     cell_texts.append((node.text).strip())
            #         cell_text = ''.join(cell_texts)
            #         row_data.append(cell_text)
            #     table_data.append(row_data)
            # elements.append(('table', table_data))
    return elements

# 파일 경로를 지정해줍니다.
#file_path = 'your_doc_file.docx'
parsed_elements = parse_docx(file_path)

# 결과를 출력해봅니다.
for element_type, content in parsed_elements:
    if element_type == 'paragraph':
        print("Paragraph:", content)
    elif element_type == 'table':
        print("Table:")
        for row in content:
            print(row)

sdadsd


IndexError: list index out of range

In [71]:
x = ['a','b']
''.join(x)

'ab'